<div style="text-align: center;">
    <h1>Machine Learning Project</h1>
    <h2>Alzheimer Disease Classification</h2>
    


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestRegressor
from imblearn.over_sampling import SMOTE
from boruta import BorutaPy
import warnings

In [11]:
pip install boruta

In [12]:
%matplotlib inline
plt.style.use('dark_background')
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv(r"alzheimers_disease_data.csv")

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.drop(columns=['PatientID', 'DoctorInCharge'], inplace=True)

In [ ]:
categorical_features = ['Gender', 'Ethnicity', 'EducationLevel', 'Smoking',
    'FamilyHistoryAlzheimers', 'CardiovascularDisease', 'Diabetes', 'Depression',
    'HeadInjury', 'Hypertension', 'MemoryComplaints', 'BehavioralProblems', 'Confusion',
    'Disorientation', 'PersonalityChanges', 'DifficultyCompletingTasks', 'Forgetfulness'
]

In [ ]:
continuous_features = ['Age', 'BMI', 'AlcoholConsumption', 'PhysicalActivity', 'DietQuality',
    'SleepQuality', 'SystolicBP', 'DiastolicBP','CholesterolTotal', 'CholesterolLDL',
    'CholesterolHDL','CholesterolTriglycerides', 'MMSE', 'FunctionalAssessment','ADL'
]

In [ ]:
class CategoricalPlotter:
    def __init__(self, data, target_column):
        """
        Initialize the CategoricalPlotter with the dataset and target column.

        Parameters:
        - data: pandas DataFrame containing both features and the target column.
        - target_column: string, name of the column to be used as the target variable.
        """
        self.data = data
        self.target_column = target_column
        self.fig, self.axes = plt.subplots(
            nrows=6, ncols=3, figsize=(18, 24)
        )
        self.axes = self.axes.flatten()

    def plot_categorical(self, idx, feature):
        """
        Plot a countplot for a given categorical feature against the target column.

        Parameters:
        - idx: index of the subplot in the grid.
        - feature: string, name of the categorical feature to plot.
        """
        ax = self.axes[idx]
        sns.countplot(
            data=self.data, x=feature, hue=self.target_column, ax=ax, palette='bright', edgecolor='white', linewidth=2
        )
        ax.set_title(
            f'{feature.capitalize()} vs {self.target_column.capitalize()}', fontsize=12, fontweight='bold'
        )
        ax.set_ylabel('Count', fontsize=10)
        ax.set_xlabel(feature.capitalize(), fontsize=10)
        ax.grid(True, linestyle='--', alpha=0.7)
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

    def show_categorical_plots(self, categorical_features):
        """
        Generate and display the plots for all categorical features.

        Parameters:
        - categorical_features: list of strings, names of the categorical features to plot.
        """
        for idx, feature in enumerate(categorical_features):
            if idx < len(self.axes):
                self.plot_categorical(idx, feature)
        for idx in range(len(categorical_features), len(self.axes)):
            self.fig.delaxes(self.axes[idx])
        plt.tight_layout()
        plt.subplots_adjust(hspace=0.4, wspace=0.4)
        plt.show()

In [ ]:
plotter = CategoricalPlotter(df, target_column='Diagnosis')
plotter.show_categorical_plots(categorical_features)

In [ ]:
class ContinuousPlotter:
    def __init__(self, data):
        """
        Initialize the ContinuousPlotter with the dataset.

        Parameters:
        - data: pandas DataFrame containing continuous features.
        """
        self.data = data
        self.fig, self.axes = plt.subplots(
            nrows=5, ncols=3, figsize=(18, 24)
        )
        self.axes = self.axes.flatten()

    def plot_distribution(self, idx, feature):
        """
        Plot the distribution for a given continuous feature.

        Parameters:
        - idx: index of the subplot in the grid.
        - feature: string, name of the continuous feature to plot.
        """
        ax = self.axes[idx]
        sns.histplot(
            self.data[feature], ax=ax, color='blue', bins=30, stat='density'
        )
        sns.kdeplot(
            self.data[feature], ax=ax, linestyle='-', color='red', linewidth=2
        )
        ax.set_title(
            f'Distribution of {feature.capitalize()}', fontsize=12, fontweight='bold'
        )
        ax.set_ylabel('Frequency', fontsize=10)
        ax.set_xlabel(feature.capitalize(), fontsize=10)
        ax.grid(True, linestyle='--', alpha=0.7)
    def show_distributions(self, continuous_features):
        """
        Generate and display the distribution plots for all continuous features.

        Parameters:
        - continuous_features: list of strings, names of the continuous features to plot.
        """
        for idx, feature in enumerate(continuous_features):
            if idx < len(self.axes):
                self.plot_distribution(idx, feature)
        for idx in range(len(continuous_features), len(self.axes)):
            self.fig.delaxes(self.axes[idx])
        plt.tight_layout()
        plt.subplots_adjust(hspace=0.4, wspace=0.4)
        plt.show()

In [ ]:
plotter = ContinuousPlotter(df)
plotter.show_distributions(continuous_features)

#### Splitting the Data into Train, Validation, and Test Data.

In [ ]:
X = df.drop(columns=['Diagnosis'])
Y = df['Diagnosis']

> Splitting into Train and Validation sets.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, Y, test_size=0.3, stratify=Y, shuffle=True, random_state=42
)

> Splitting Validation set into Validation and Test sets.

In [ ]:
X_val, X_test, y_val, y_test = train_test_split(
    X_val, y_val, test_size=0.3, stratify=y_val, shuffle=True, random_state=42
)

#### Handling Outlier values.

In [ ]:
class BoxPlotter:
    def __init__(self, X, Y):
        self.X_train = X
        self.y_train = Y.astype('category')
        self.fig, self.axes = plt.subplots(
            nrows=5, ncols=3, figsize=(18, 20)
        )

    def plot_box(self, row, col, x):
        ax = self.axes[row, col]
        sns.boxplot(
            data=self.X_train, x=x, y=self.y_train, ax=ax, palette='bright', linecolor='white', linewidth=2
        )
        ax.set_title(
            f'{x.capitalize()} vs Diagnosis', fontsize=12, fontweight='bold'
        )
        ax.set_ylabel('Diagnosis', fontsize=10)
        ax.set_xlabel(x.capitalize(), fontsize=10)
        ax.grid(
            True, linestyle='--', alpha=0.7
        )
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

    def show_box_plots(self):
        for idx in range(len(continuous_features), 15):
            self.fig.delaxes(self.axes.flatten()[idx])
        plt.tight_layout()
        plt.subplots_adjust(hspace=0.4, wspace=0.4)
        plt.show()

In [ ]:
plot = BoxPlotter(X_train, y_train)

for idx, col in enumerate(continuous_features):
    plot.plot_box(idx // 3, idx % 3, x=col)

plot.show_box_plots()

In [ ]:
result_df = pd.DataFrame(columns=['Features', 'Category', 'Count'])

for col in categorical_features:
    value_counts = df[col].value_counts().reset_index()
    value_counts.columns = ['Category', 'Count']
    value_counts['Features'] = col
    result_df = pd.concat([result_df, value_counts], ignore_index=True)

grouped_df = result_df.groupby('Features').agg({'Category': list, 'Count': list}).reset_index()
grouped_df

#### Balancing Training set.

In [ ]:
def plot_data_distribution(X, y, title, ax):
    sns.countplot(x=y, palette='bright', ax=ax, edgecolor='white', linewidth=2)
    for p in ax.patches:
        count = p.get_height()
        total = len(y)
        percentage = 100 * count / total
        ax.annotate(
            f'{count} ({percentage:.1f}%)',
            (p.get_x() + p.get_width() / 2., p.get_height()),
            ha='center', va='bottom', fontsize=9, color='black'
        )
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_xlabel('Diagnosis', fontsize=12)
    ax.set_ylabel('Count', fontsize=12)
    ax.grid(axis='y', linestyle='--', alpha=0.7)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

plot_data_distribution(X_train, y_train, "Original Data", axes[0])

X_train, y_train = SMOTE(random_state=42).fit_resample(X_train, y_train)

plot_data_distribution(X_train, y_train, "SMOTE Resampled Data", axes[1])

plt.tight_layout()
plt.show()

#### Feature Selection.

> Correlation Analysis.

In [ ]:
train_copy = pd.concat(
    [X_train, y_train,], axis=1
)

corr_matrix = train_copy.corr()

plt.figure(figsize=(30, 30))
sns.heatmap(
    corr_matrix, cmap='Spectral', annot=True, fmt='.2f', linewidths=2, linecolor='black'
)
plt.title(
    'Correlation Matrix', fontsize=20, fontweight='bold'
)
plt.show()

> Mutual Information.

In [ ]:
feature_zero_count = {}

for X, Y in zip([X_train, X_val, X_test], [y_train, y_val, y_test]):
    feature_names = X.columns
    scores = mutual_info_classif(X, Y)
    indices = np.argsort(scores)[::-1]
    sorted_scores = scores[indices]
    sorted_feature_names = [feature_names[i] for i in indices]

    for feature, score in zip(feature_names, scores):
        if feature not in feature_zero_count:
            feature_zero_count[feature] = 0
        if score == 0:
            feature_zero_count[feature] += 1

    plt.figure(figsize=(10, 5))
    plt.bar(range(len(sorted_scores)), sorted_scores)
    plt.xticks(
        range(len(sorted_scores)), sorted_feature_names, rotation='vertical'
    )
    plt.title('Mutual Information Scores')
    plt.ylabel('Score')
    plt.xlabel('Features')
    plt.tight_layout()
    plt.show()

In [ ]:
features_to_drop = [
    feature for feature, zero_count in feature_zero_count.items()
    if zero_count >= 1
]

> Boruta algorithm.

In [ ]:
rfr = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)

In [ ]:
feat_selector = BorutaPy(
    verbose=2, estimator=rfr, n_estimators='auto', max_iter=10
)

In [ ]:
feat_selector.fit(X_train.values, y_train.values)

In [ ]:
cols_to_drop = []

print("~ Support and Ranking for each feature:")
for i in range(len(feat_selector.support_)):
    if feat_selector.support_[i]:
        print("Passes the test: ", X_train.columns[i], " - Ranking: ", feat_selector.ranking_[i])
    else:
        print("Doesn't pass the test: ", X_train.columns[i], " - Ranking: ", feat_selector.ranking_[i])
        cols_to_drop.append(X_train.columns[i])

In [ ]:
print('~ Features to drop:\n')
print(f'Based on Mutual information scores: {features_to_drop}\n')
print(f'Based on Boruta algorithm: {cols_to_drop}')

In [ ]:
X_train.drop(columns=cols_to_drop, inplace=True)
X_val.drop(columns=cols_to_drop, inplace=True)
X_test.drop(columns=cols_to_drop, inplace=True)

In [ ]:
# Assuming 'selected_features' is the set of features chosen by Boruta
X_train_selected = X_train

svm = SVC()
svm.fit(X_train_selected, y_train)

In [ ]:
import shap

explainer = shap.KernelExplainer(svm.predict, X_train_selected)

shap_values = explainer.shap_values(X_train_selected[:100])

shap.summary_plot(shap_values, X_train_selected[:100], feature_names=X_train_selected.columns)

shap.force_plot(explainer.expected_value, shap_values[0], X_train_selected.iloc[0], feature_names=X_train_selected.columns)


Feature Scaling

In [ ]:
scaler = StandardScaler()

scaled_X_train = scaler.fit_transform(X_train)
scaled_X_val = scaler.transform(X_val)
scaled_X_test = scaler.transform(X_test)

#### Feature Extraction.

In [ ]:
pca = PCA()
pca.fit(scaled_X_train)

explained_variance = np.cumsum(pca.explained_variance_ratio_)

n_components_95 = np.argmax(explained_variance >= 0.95) + 1

print(f"Number of components for 95% variance: {n_components_95}")

plt.figure(figsize=(8, 5))
plt.plot(range(1, len(explained_variance) + 1), explained_variance, marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance vs. Number of Components')
plt.grid()
plt.show()

In [ ]:
pca = PCA(n_components=n_components_95)

train_reduced = pca.fit_transform(scaled_X_train)
val_reduced = pca.transform(scaled_X_val)
test_reduced = pca.transform(scaled_X_test)

In [ ]:
svc = SVC(kernel='rbf', C=1, random_state=42)
svc.fit(train_reduced, y_train)

y_pred_val = svc.predict(val_reduced)

accuracy_val = accuracy_score(y_val, y_pred_val)
print(f'Validation Accuracy: {accuracy_val:.4f}')

print("\nClassification Report (Validation):")
print(classification_report(y_val, y_pred_val))

In [ ]:
params = {
    'C': [0.001,0.01, 0.1, 1, 10, 100],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': [2, 3, 4, 5],
    'gamma': ['scale', 'auto']
}

In [ ]:
grid_search = GridSearchCV(
    SVC(), params, cv=5, scoring='accuracy'
)
grid_search.fit(train_reduced, y_train)

In [ ]:
print("Best parameters:", grid_search.best_params_)

In [ ]:
res = grid_search.predict(val_reduced)

In [ ]:
print(accuracy_score(y_val, res))

In [ ]:
print(classification_report(y_val, res))

In [ ]:
pca = PCA(n_components=2)
X_train_2d = pca.fit_transform(train_reduced)

best_params = grid_search.best_params_

svm = SVC(**best_params)
svm.fit(X_train_2d, y_train)

x_min, x_max = X_train_2d[:, 0].min() - 1, X_train_2d[:, 0].max() + 1
y_min, y_max = X_train_2d[:, 1].min() - 1, X_train_2d[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.05),
                     np.arange(y_min, y_max, 0.05))

Z = svm.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.figure(figsize=(10, 6))
plt.contourf(xx, yy, Z, alpha=0.8, cmap=plt.cm.coolwarm_r)
plt.scatter(X_train_2d[:, 0], X_train_2d[:, 1], c=y_train, cmap=plt.cm.coolwarm_r, edgecolor='black', s=15, linewidths=0.6)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('SVM Decision Boundary with Best Parameters')
plt.show()

In [ ]:
import shap

explainer = shap.KernelExplainer(svm.predict, X_train_2d)
# limting to 100 for faster computation
shap_values = explainer.shap_values(X_train_2d[:100])

shap.summary_plot(shap_values, X_train_2d[:100])

shap.force_plot(explainer.expected_value, shap_values[0], X_train_2d[0])